In [117]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import torch.utils.data as data
import torchvision.transforms as transforms

# SciKit
from sklearn.model_selection import train_test_split

# Python
import pandas as pd
import numpy as np
import time

# Graphing
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


# utils
from utils import *

# Device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [118]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [119]:
# ML Parameters
lr = 1e-3
min_lr = 1e-5
epochs = 100
batch_size = 512

# Data Parameters
data_seq_len = 100
data_n_features = 1
data_embedding_dim = 64

In [120]:
#load train data
df_sensorA_normal = pd.read_csv('data/sensor_A_normal.csv', header=None)
df_sensorB_normal = pd.read_csv('data/sensor_B_normal.csv', header=None)
df_sensorC_normal = pd.read_csv('data/sensor_C_normal.csv', header=None)
df_sensorD_normal = pd.read_csv('data/sensor_D_normal.csv', header=None)
df_sensorE_normal = pd.read_csv('data/sensor_E_normal.csv', header=None)
print(len(df_sensorA_normal))
df_train = [df_sensorA_normal, df_sensorB_normal, df_sensorC_normal, df_sensorD_normal, df_sensorE_normal]

# load val data
df_sensorA_public = pd.read_csv('data/sensor_A_public.csv', header=None)
df_sensorB_public = pd.read_csv('data/sensor_B_public.csv', header=None)
df_sensorC_public = pd.read_csv('data/sensor_C_public.csv', header=None)
df_sensorD_public = pd.read_csv('data/sensor_D_public.csv', header=None)
df_sensorE_public = pd.read_csv('data/sensor_E_public.csv', header=None)
print(len(df_sensorA_public))
df_test = [df_sensorA_public, df_sensorB_public, df_sensorC_public, df_sensorD_public, df_sensorE_public]

# load test data
df_sensorA_private = pd.read_csv('data/sensor_A_private.csv', header=None)
df_sensorB_private = pd.read_csv('data/sensor_B_private.csv', header=None)
df_sensorC_private = pd.read_csv('data/sensor_C_private.csv', header=None)
df_sensorD_private = pd.read_csv('data/sensor_D_private.csv', header=None)
df_sensorE_private = pd.read_csv('data/sensor_E_private.csv', header=None)
print(len(df_sensorA_private))
df_private = [df_sensorA_private, df_sensorB_private, df_sensorC_private, df_sensorD_private, df_sensorE_private]

2876
4001
4001


In [121]:
for df in df_test:
    df.drop(df.index[-1], inplace=True)
    df = df[:401]

In [122]:
class SensorDataset(data.Dataset):
    """
        Support class for the loading and batching of sequences of samples

        Args:
            dataset (Tensor): Tensor containing all the samples
            sequence_length (int): length of the analyzed sequence by the LSTM
            transforms (object torchvision.transform): Pytorch's transforms used to process the data
    """
    ##  Constructor
    def __init__(self, df, seq_len=1, transform=None):
        self.dataset = df
        self.seq_len = seq_len
        self.transforms = transform

    ##  Override total dataset's length getter
    def __len__(self):
        return self.dataset.__len__()

    ##  Override single items' getter
    def __getitem__(self, idx):
        if idx + self.seq_len > len(self.dataset):
            if self.transforms is not None:    
                item = torch.zeros(self.seq_len, self.dataset[0].__len__())
                item[:self.__len__()-idx] = self.transform(self.dataset[idx:])
                return item, item
            else:
                item = []
                item[:self.__len__()-idx] = self.dataset[idx:]
                return item, item

        else:
            if self.transforms is not None:
                return self.transforms(self.dataset[idx:idx+self.seq_len]), self.transforms(self.dataset[idx:idx+self.seq_len])
            else:
                return self.dataset[idx:idx+self.seq_len], self.dataset[idx:idx+self.seq_len]

                
# Helper for transforming the data from a list to Tensor
def listToTensor(list):
    tensor = torch.empty(list.__len__(), list[0].__len__())
    for i in range(list.__len__()):
        tensor[i, :] = torch.from_numpy(list[i])
    return tensor    

In [123]:
# transform
data_transform = transforms.Lambda(lambda x: listToTensor(x))
# Dataset Objects
train_dataset = []
for df in df_train:
    data = np.array(df.iloc[1:, 0].values).astype(float).reshape(-1, 1)
    train_dataset.append(SensorDataset(data, seq_len=data_seq_len, transform = data_transform))
val_dataset = []
for df in df_test:
    data = np.array(df.iloc[1:, 0].values).astype(float).reshape(-1, 1)
    val_dataset.append(SensorDataset(data, seq_len=data_seq_len, transform = data_transform))
private_dataset = []
for df in df_private:
    data = np.array(df.iloc[1:, 0].values).astype(float).reshape(-1, 1)
    private_dataset.append(SensorDataset(data, seq_len=data_seq_len, transform = data_transform))

# Pytorch DataLoader objects
train_loader = []
for dataset in train_dataset:
    train_loader.append(DataLoader(dataset, batch_size=batch_size, shuffle=False))
val_loader = []
for dataset in val_dataset:
    val_loader.append(DataLoader(dataset, batch_size=batch_size, shuffle=False))
private_loader = []
for dataset in private_dataset:
    private_loader.append(DataLoader(dataset, batch_size=batch_size, shuffle=False))

ValueError: could not convert string to float: 'telemetry'

In [ ]:
class LSTMEncoder(nn.Module):
    
    def __init__(self, seq_len, n_features, embedding_dim):
        super(LSTMEncoder, self).__init__()
        
        # Parameters
        self.seq_len = seq_len
        self.n_features = n_features
        self.embedding_dim = embedding_dim
        self.hidden_dim = 2*embedding_dim
        
        # Neural Network Layers
        self.lstm1 = nn.LSTM(self.n_features, self.hidden_dim, num_layers=1, batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_dim, self.embedding_dim, num_layers=1, batch_first=True)
    
    def forward(self, i): 
        i, _ = self.lstm1(i)               # from (batch, seq_len, n_features) to (batch, seq_len, hidden_dim)
        i, (hidden_n, _) = self.lstm2(i)   # from (batch, seq_len, hidden_dim) to (batch, seq_len, embedding_dim)
        return hidden_n                    # hidden_n shape: (num_layers*num_directions, batch, embedding_dim)


class LSTMDecoder(nn.Module):

    def __init__(self, seq_len, embedding_dim, n_features=1):
        super(LSTMDecoder, self).__init__()

        # Parameters
        self.seq_len = seq_len
        self.embedding_dim = embedding_dim
        self.hidden_dim = 2*embedding_dim
        self.n_features = n_features
        
        # Neural Network Layers
        self.lstm1 = nn.LSTM(self.embedding_dim, self.embedding_dim, num_layers=1, batch_first=True)
        self.lstm2 = nn.LSTM(self.embedding_dim, self.hidden_dim, num_layers=1, batch_first=True)
        self.output_layer = nn.Linear(self.hidden_dim, n_features)
        
    def forward(self, i):
        # Do padding
        i = i.repeat(self.seq_len, 1, 1)                       # repeat (1, embedding_dim) to (seq_len, embedding_dim)
        i = i.reshape((-1, self.seq_len, self.embedding_dim))  # reshape to (batch, seq_len, embedding_dim)
        
        # Traverse neural layers
        i, _ = self.lstm1(i)      # from (batch, seq_len, embedding_dim) to (batch, seq_len, embedding_dim)
        i, _ = self.lstm2(i)      # from (batch, seq_len, embedding_dim) to (batch, seq_len, hidden_dim)
        i = self.output_layer(i)  # from (batch, seq_len, hidden_dim) to (batch, seq_len, n_features)
        
        return i


class LSTMAutoencoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=64):
        super(LSTMAutoencoder, self).__init__()
        self.encoder = LSTMEncoder(seq_len, n_features, embedding_dim).to(device)
        self.decoder = LSTMDecoder(seq_len, embedding_dim, n_features).to(device)
        
    def forward(self, i):
        i = self.encoder(i)
        i = self.decoder(i)
        return i

In [ ]:
# Define model1 for E1
model = LSTMAutoencoder(data_seq_len, data_n_features, data_embedding_dim)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
lr_schedule = cosine_scheduler(
        lr,
        min_lr,
        epochs, len(train_loader[0]),
        warmup_epochs=10,
    )

In [ ]:
def training(model, optimizer, trainset_iterator, validationset_iterator, epoch):
    train_losses, test_losses = [], []
    criterion = nn.L1Loss(reduction='sum').to(device)

    for epoch in range(epoch):
        
        print("Epoch %d training started ..." % epoch)
        start_time = time.time()
        
        # Enter Train Mode
        model.train()     
        train_loss = 0
        for it, ii in enumerate(trainset_iterator):
            ii = ii.to(device)              # move to GPU if necessary
            it = len(trainset_iterator) * epoch + it  # global training iteration
            for i, param_group in enumerate(optimizer.param_groups):
                param_group["lr"] = lr_schedule[it]
            optimizer.zero_grad()           # generate prediction
            preds = model(ii)               # generate prediction
            loss = criterion(preds, ii)     # calculate loss
            loss.backward()                 # back propagation of gradients and update weights
            optimizer.step()                # update optimizer
            train_loss += loss.item()       # record training losses

        # Enter Validation Mode
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for ii in validationset_iterator:
                ii = ii.to(device)          # move to GPU if necessary
                preds = model(ii)           # generate prediction
                loss = criterion(preds, ii) # calculate loss
                test_loss += loss           # record validation testing losses
        
        end_time = time.time()
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        print("Epoch %d completed - train_loss: %f , test_loss: %f" % (epoch, train_loss, test_loss))
        print("Epoch %d training time: %f" %(epoch, (end_time - start_time)))
    
    return train_losses, test_losses

In [ ]:
def predict(model, dataset_iterator):
    preds, losses = np.array([]), np.array([])
    criterion = nn.L1Loss(reduction='none').to(device)
    
    # Enter Validation Mode
    model = model.eval()
    with torch.no_grad():
        for ii in dataset_iterator:
            
            # move to GPU if necessary
            ii = ii.to(device)
            
            # generate prediction
            pred = model(ii)
            
            # calculate loss
            loss = criterion(pred, ii)
            
            # record predictions
            preds = np.append(preds, pred.cpu().numpy())
            
            # record mean loss of each sample
            loss = loss.reshape((-1, data_seq_len)).numpy()  # from (batch, seq_len, n_feature) to (batch, seq_len)
            losses = np.append(losses, [np.sum(i) for i in loss])  # sum of all seq_len losses into one loss
    
    preds = preds.reshape((-1, data_seq_len))  # reshape to (batch, seq_len)
    return preds, losses

In [ ]:
for i in range(5):
    # Training:
    train_losses, test_losses = training(model, optimizer, train_loader[i], val_loader[i], epochs)

    # Saving trained models
    torch.save(model, f'./sensor_model_{i}.pth')

Epoch 0 training started ...


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.